In [2]:
import io
import open3d as o3d
import scipy.io as sio
import numpy as np
import numpy.linalg as LA
import copy
import random

In [11]:
path = 'Home_002_1/image_structs.mat'
data = sio.loadmat(path)

In [12]:
"""
From the image_structs.mat file make a dictionary 
called output(image number: (x,z,R,t,s))
"""
xyz = data['image_structs']['world_pos']
image_names = data['image_structs']['image_name'][0]
rot = data['image_structs']['R'][0] 
scale = data['scale'][0][0]
#translation = -1*data['image_structs']['t'][0]
translation = xyz[0]
n = xyz.shape[1]
x = [xyz[0][i][0][0] for i in range(n)]
z = [xyz[0][i][2][0] for i in range(n)]
names = [name[0][:-4] for name in image_names]
if len(names) != len(x):
    raise ValueError('number of image names are not equal to the number of '
                 'poses {} != {}'.format(len(names), len(x)))
output = {}
valid_images = []
for i in range(n):
    #For some images, there's no translation matrix provided.
    if rot[i].shape[0] != 0:
        assert rot[i].shape[0] == 3
        assert rot[i].shape[1] == 3 
        valid_images.append(i)
        
        #translation[i][0] = translation[i][0] * -1
        translation[i][1] = translation[i][1] * -1
        translation[i][2] = translation[i][2] * -1
            
        output[i] = (x[i], z[i], rot[i], translation[i]*(scale)/1000, scale)
        #output[i] = (x[i], z[i], rot[i], xyz[0][i]*scale/1000, scale)
        #output[i] = (x[i], z[i], LA.inv(rot[i]), np.matmul((-1*LA.inv(rot[i])), translation[i])*scale/1000, scale)
        #output[i] = (x[i], z[i], rot[i], np.matmul((-1*LA.inv(rot[i])), translation[i])*scale/1000, scale)
    else:
        continue
        #translation[i][0] = translation[i][0] * -1
        #translation[i][1] = translation[i][1] * -1
        #translation[i][2] = translation[i][2] * -1
        output[i] = (x[i], z[i], None, translation[i]*(scale)/1000, scale)
    

In [13]:
def transformation_matrix(R, t):
    """
    Make Transformation matrix T, from rotation matrix R and translation matrix t
    """
    
    T = np.empty((4,4))
    T[:3,:3] = R
    try:
        T[:3,3] = t.reshape(3)
    except:
        T[:3,3] = None
    T[3:,] = [0,0,0,1]
    return T

In [14]:
transformation_matrices = {}

In [15]:
#For each image, make the transformation matrix from pose
for i in valid_images:
    R = output[i][2]
    t = output[i][3]
    T = transformation_matrix(R,t)
    transformation_matrices[i] = T

In [18]:
random.seed(123)
random.shuffle(valid_images)

In [19]:
#Read all/some of the point clouds
#pcds = []
path = './pc_to_wc'
if os.path.isdir(path) == False:
    os.mkdir(path)
for i in valid_images:
    pcd = o3d.io.read_point_cloud(f'./point_clouds/{i}.pcd')
    pcd.voxel_down_sample(voxel_size=0.05)
    
    pcd.transform(transformation_matrices[i])
    #pcds.append(pcd)
    o3d.io.write_point_cloud(f"{path}/{i}.pcd", pcd)      

In [20]:
o3d.visualization.draw_geometries(pcds)

In [9]:
"""
Calculate the relative transformation matrix, which will transform the point clouds
from any frame to the first frame

path = './pc_to_wc'
for i in range(len(pcds)):
    #if i != 0:
    #    T = np.matmul(transformation_matrices[i], LA.inv(transformation_matrices[0]))
    #    pcds[i].transform(T)
        #pcds[i].transform(transformation_matrices[i])
    pcds[i].transform(transformation_matrices[i])
    #o3d.io.write_point_cloud(f"{path}/{i+200}.pcd", pcds[i])     
"""